In [1]:
import math

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.nn import TransformerEncoder, TransformerEncoderLayer


In [3]:
ntokens = 5000 #len(vocab) # 단어 사전(어휘집)의 크기
emsize = 200 # 임베딩 차원
nhid = 200 # nn.TransformerEncoder 에서 피드포워드 네트워크(feedforward network) 모델의 차원
nlayers = 2 # nn.TransformerEncoder 내부의 nn.TransformerEncoderLayer 개수
nhead = 2 # 멀티헤드 어텐션(multi-head attention) 모델의 헤드 개수
dropout = 0.2 # 드랍아웃(dropout) 값

In [4]:
# TransformerModel(ntokens, emsize, nhead, nhid, nlayers, dropout)
d_model = emsize
max_len=5000
pe = torch.zeros(max_len, d_model)

### 기본적으로 저 pe라는 텐서를 만드는게 목적
- 최초 정의를 max_len으로 하는 이유는 모델 최대크기 사용에서도 유효하길 바래서라는 생각
- 여기서 d_model은 embeding의 dim

In [7]:
print(pe)
print(pe.shape)
print(pe.dtype)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]])
torch.Size([5000, 200])
torch.float32


In [22]:
position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)

### position
- token의 위치정보에 따라 선행 계산된 값을 전달하기 위해서 미리 포지션 matrix를 생성
- arange(start, end, type) : range의 torch 버전
- unsqueeze(dim) : shape가 [5000]인 애한테 unsqueeze(0)해주면 [1, 5000]이 되고 unsqueeze(1)해주면 [5000, 1]이 되는 식으로 dim에다가 1인 차원을 추가해줌, 참고로 dim이 1인 차원을 모두 날려버리는 squeeze()가 있다.

In [23]:
print(position)
print(position.shape)
print(position.dtype)

tensor([[0.0000e+00],
        [1.0000e+00],
        [2.0000e+00],
        ...,
        [4.9970e+03],
        [4.9980e+03],
        [4.9990e+03]])
torch.Size([5000, 1])
torch.float32


### div_term
- 사실 이건그냥 주기다.
  - dim마다 같은 값을 더하는게 아니라 주기를 다르게 해서 조금 더 티나게 만들려는듯 굳이 다르게 해야하는가는 잘 모르겠다.
  - 참고로 간격이 dim이 원래의 절반이 되게 arange간격을 2로 설정되어있다.
    - 나중에 cos랑 sin을 반반씩 섞어서 매핑할거라 반만쓴다.
    - 그래서 전체 dim은 짝수여야만 한다.

In [30]:
div_term = (torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0) / d_model))).unsqueeze(0)

In [31]:
print(div_term)
print(div_term.shape)
print(div_term.dtype)

tensor([[1.0000e+00, 9.1201e-01, 8.3176e-01, 7.5858e-01, 6.9183e-01, 6.3096e-01,
         5.7544e-01, 5.2481e-01, 4.7863e-01, 4.3652e-01, 3.9811e-01, 3.6308e-01,
         3.3113e-01, 3.0200e-01, 2.7542e-01, 2.5119e-01, 2.2909e-01, 2.0893e-01,
         1.9055e-01, 1.7378e-01, 1.5849e-01, 1.4454e-01, 1.3183e-01, 1.2023e-01,
         1.0965e-01, 1.0000e-01, 9.1201e-02, 8.3176e-02, 7.5858e-02, 6.9183e-02,
         6.3096e-02, 5.7544e-02, 5.2481e-02, 4.7863e-02, 4.3652e-02, 3.9811e-02,
         3.6308e-02, 3.3113e-02, 3.0200e-02, 2.7542e-02, 2.5119e-02, 2.2909e-02,
         2.0893e-02, 1.9055e-02, 1.7378e-02, 1.5849e-02, 1.4454e-02, 1.3183e-02,
         1.2023e-02, 1.0965e-02, 1.0000e-02, 9.1201e-03, 8.3176e-03, 7.5858e-03,
         6.9183e-03, 6.3096e-03, 5.7544e-03, 5.2481e-03, 4.7863e-03, 4.3652e-03,
         3.9811e-03, 3.6308e-03, 3.3113e-03, 3.0200e-03, 2.7542e-03, 2.5119e-03,
         2.2909e-03, 2.0893e-03, 1.9055e-03, 1.7378e-03, 1.5849e-03, 1.4454e-03,
         1.3183e-03, 1.2023e

In [41]:
a = div_term * position 
print(a)
print(a.shape) # 당연한거지만 매트릭스 연산에서 앞에거의 앞의 dim 뒤에꺼의 뒤의 dim이 남는다. 하지만 * 연산은 matrix의 요소별 곱이다
print(a.dtype)

tensor([[0.0000e+00, 0.0000e+00, 0.0000e+00,  ..., 0.0000e+00, 0.0000e+00,
         0.0000e+00],
        [1.0000e+00, 9.1201e-01, 8.3176e-01,  ..., 1.3183e-04, 1.2023e-04,
         1.0965e-04],
        [2.0000e+00, 1.8240e+00, 1.6635e+00,  ..., 2.6365e-04, 2.4045e-04,
         2.1930e-04],
        ...,
        [4.9970e+03, 4.5573e+03, 4.1563e+03,  ..., 6.5873e-01, 6.0077e-01,
         5.4791e-01],
        [4.9980e+03, 4.5582e+03, 4.1572e+03,  ..., 6.5886e-01, 6.0089e-01,
         5.4802e-01],
        [4.9990e+03, 4.5591e+03, 4.1580e+03,  ..., 6.5900e-01, 6.0101e-01,
         5.4813e-01]])
torch.Size([5000, 100])
torch.float32


In [43]:
div2 = torch.cat([div_term,div_term], 0)
print(div2)
print(div2.shape)
print(div2.dtype)

tensor([[1.0000e+00, 9.1201e-01, 8.3176e-01, 7.5858e-01, 6.9183e-01, 6.3096e-01,
         5.7544e-01, 5.2481e-01, 4.7863e-01, 4.3652e-01, 3.9811e-01, 3.6308e-01,
         3.3113e-01, 3.0200e-01, 2.7542e-01, 2.5119e-01, 2.2909e-01, 2.0893e-01,
         1.9055e-01, 1.7378e-01, 1.5849e-01, 1.4454e-01, 1.3183e-01, 1.2023e-01,
         1.0965e-01, 1.0000e-01, 9.1201e-02, 8.3176e-02, 7.5858e-02, 6.9183e-02,
         6.3096e-02, 5.7544e-02, 5.2481e-02, 4.7863e-02, 4.3652e-02, 3.9811e-02,
         3.6308e-02, 3.3113e-02, 3.0200e-02, 2.7542e-02, 2.5119e-02, 2.2909e-02,
         2.0893e-02, 1.9055e-02, 1.7378e-02, 1.5849e-02, 1.4454e-02, 1.3183e-02,
         1.2023e-02, 1.0965e-02, 1.0000e-02, 9.1201e-03, 8.3176e-03, 7.5858e-03,
         6.9183e-03, 6.3096e-03, 5.7544e-03, 5.2481e-03, 4.7863e-03, 4.3652e-03,
         3.9811e-03, 3.6308e-03, 3.3113e-03, 3.0200e-03, 2.7542e-03, 2.5119e-03,
         2.2909e-03, 2.0893e-03, 1.9055e-03, 1.7378e-03, 1.5849e-03, 1.4454e-03,
         1.3183e-03, 1.2023e

In [44]:
a = div2 * position 
print(a)
print(a.shape) #
print(a.dtype)

RuntimeError: The size of tensor a (2) must match the size of tensor b (5000) at non-singleton dimension 0

In [46]:
pe[:, 0::2] = torch.sin(position * div_term)
pe[:, 1::2] = torch.cos(position * div_term)

print(pe)
print(pe.shape)
print(pe.dtype)

tensor([[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
          0.0000e+00,  1.0000e+00],
        [ 8.4147e-01,  5.4030e-01,  7.9074e-01,  ...,  1.0000e+00,
          1.0965e-04,  1.0000e+00],
        [ 9.0930e-01, -4.1615e-01,  9.6811e-01,  ...,  1.0000e+00,
          2.1930e-04,  1.0000e+00],
        ...,
        [ 9.5625e-01, -2.9254e-01,  9.0551e-01,  ...,  8.2490e-01,
          5.2090e-01,  8.5362e-01],
        [ 2.7050e-01, -9.6272e-01,  2.1917e-01,  ...,  8.2483e-01,
          5.2100e-01,  8.5356e-01],
        [-6.6395e-01, -7.4778e-01, -6.3742e-01,  ...,  8.2476e-01,
          5.2109e-01,  8.5350e-01]])
torch.Size([5000, 200])
torch.float32


### 아래의 1인 차원을 하나 추가해주는데 이거 의미는 이따 정리

In [47]:
pe = pe.unsqueeze(0).transpose(0, 1)

print(pe)
print(pe.shape)
print(pe.dtype)

tensor([[[ 0.0000e+00,  1.0000e+00,  0.0000e+00,  ...,  1.0000e+00,
           0.0000e+00,  1.0000e+00]],

        [[ 8.4147e-01,  5.4030e-01,  7.9074e-01,  ...,  1.0000e+00,
           1.0965e-04,  1.0000e+00]],

        [[ 9.0930e-01, -4.1615e-01,  9.6811e-01,  ...,  1.0000e+00,
           2.1930e-04,  1.0000e+00]],

        ...,

        [[ 9.5625e-01, -2.9254e-01,  9.0551e-01,  ...,  8.2490e-01,
           5.2090e-01,  8.5362e-01]],

        [[ 2.7050e-01, -9.6272e-01,  2.1917e-01,  ...,  8.2483e-01,
           5.2100e-01,  8.5356e-01]],

        [[-6.6395e-01, -7.4778e-01, -6.3742e-01,  ...,  8.2476e-01,
           5.2109e-01,  8.5350e-01]]])
torch.Size([5000, 1, 200])
torch.float32
